<a href="https://colab.research.google.com/github/jitendra-121/gi/blob/main/DR2Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn import svm
from sklearn.metrics import accuracy_score

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import numpy as np
import cv2
import os
import pandas as pd



data_dir='/content/drive/MyDrive/ ML data/Colored Images'
no_dir = os.path.join(data_dir, "No_DR")
proliferate_dir = os.path.join(data_dir, "Proliferate_DR")


# Assuming all images are of the same size (e.g., 100x100)
image_size = (150, 150)




In [3]:
# Function to load and preprocess images
def load_images_no_dr(directory):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".png"):
            img = cv2.imread(os.path.join(directory, filename)) #read a specific image from the directory
            img = cv2.resize(img, image_size)#
            images.append(img.flatten())  # Flatten image into 1D array
            labels.append(0)  # 0 for cats, 1 for dogs
    return images, labels

# Load and preprocess images
no_images, no_labels = load_images_no_dr(no_dir)



In [4]:
def load_images_proliferate_dr(directory):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith(".png"):
            img = cv2.imread(os.path.join(directory, filename)) #read a specific image from the directory
            img = cv2.resize(img, image_size)#
            images.append(img.flatten())  # Flatten image into 1D array
            labels.append(1)  # 0 for cats, 1 for dogs
    return images, labels

# Load and preprocess images
proliferate_images, proliferate_labels = load_images_proliferate_dr(proliferate_dir)

In [5]:
# Combine cat and dog images and labels
images = np.concatenate((no_images,proliferate_images), axis=0)
labels = np.concatenate((no_labels,proliferate_labels), axis=0)

# Convert images and labels to pandas DataFrames
image_df = pd.DataFrame(images)
label_df = pd.DataFrame(labels, columns=["label"])

# Combine image features and labels into a single DataFrame
df = pd.concat([image_df, label_df], axis=1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3662 entries, 0 to 3661
Columns: 67501 entries, 0 to label
dtypes: int64(1), uint8(67500)
memory usage: 235.8 MB


In [7]:
df.head(50)

,0,1,2,3,4,5,6,7,8,9,...,67491,67492,67493,67494,67495,67496,67497,67498,67499,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,2,0,2,3,0,1,2,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,2,0,2,3,1,0,2,1,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,1,2,0,1,2,0,1,2,0,...,1,1,1,1,1,1,1,1,1,0
6,0,1,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,0,0,0,0
7,1,1,1,1,1,1,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
8,0,1,2,0,1,2,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)


# Verify the shuffling
# print(shuffled_df['category'].value_counts())

In [9]:
shuffled_df

,0,1,2,3,4,5,6,7,8,9,...,67491,67492,67493,67494,67495,67496,67497,67498,67499,label
0,5,3,4,6,4,4,6,3,5,6,...,7,3,5,6,5,5,6,3,5,1
1,2,1,3,2,1,3,2,1,3,2,...,2,1,2,1,1,1,2,1,3,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,3,0,1,2,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
4,13,7,7,14,7,9,14,7,10,13,...,13,8,10,13,8,9,14,8,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3657,0,1,2,0,1,2,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
3658,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3659,0,1,2,0,2,3,0,2,2,0,...,1,1,1,1,1,1,1,1,1,0
3660,5,3,3,6,3,4,5,3,5,6,...,6,4,4,6,4,4,6,4,4,1


In [10]:
import cudf
shuffled= cudf.DataFrame(shuffled_df)

In [11]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 490, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 490 (delta 149), reused 124 (delta 91), pack-reused 269
Receiving objects: 100% (490/490), 136.70 KiB | 764.00 KiB/s, done.
Resolving deltas: 100% (251/251), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.7 MB/s eta 0:00:00
Installing the rest of the RAPIDS 24.4.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 GB 531.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 GB 826.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.9/920.9 kB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
import cuml
import cupy as cp

In [13]:
x=shuffled.iloc[:,:-1]
y=shuffled.iloc[:,-1]

In [14]:
from cuml.preprocessing import StandardScaler
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

In [15]:
x_scaled #cudf dataframe

,0,1,2,3,4,5,6,7,8,9,...,67490,67491,67492,67493,67494,67495,67496,67497,67498,67499
0,1.032235,0.900010,0.756268,1.312019,1.378776,0.724536,1.310880,0.832545,1.135103,1.262589,...,1.270934,1.555714,0.804635,1.296567,1.270144,1.841178,1.333913,1.302940,0.879509,1.373464
1,0.010049,-0.205419,0.303521,-0.010018,-0.234176,0.278686,0.000179,-0.219121,0.265763,-0.016946,...,-0.353555,-0.006399,-0.163995,0.074528,-0.313751,-0.160126,-0.336957,0.007605,-0.143280,0.519175
2,-0.671409,-0.758133,-1.054720,-0.671037,-0.771827,-1.058862,-0.655172,-0.744954,-1.038246,-0.656714,...,-0.759677,-0.631244,-0.648311,-0.740164,-0.630530,-0.660452,-0.754675,-0.640063,-0.654674,-0.762258
3,-0.671409,0.347295,0.303521,-0.671037,-0.234176,-0.167163,-0.655172,-0.219121,-0.168906,-0.656714,...,-0.353555,-0.631244,-0.648311,-0.740164,-0.630530,-0.660452,-0.754675,-0.640063,-0.654674,-0.762258
4,3.758066,3.110867,2.114509,3.956093,2.991728,2.953783,3.932283,2.935878,3.308451,3.501775,...,2.895423,3.430250,3.226212,3.333298,3.487597,3.342157,3.004783,3.893611,3.436479,3.082041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3657,-0.671409,-0.205419,-0.149226,-0.671037,-0.234176,-0.167163,-0.655172,-0.219121,-0.168906,-0.656714,...,-0.353555,-0.631244,-0.648311,-0.740164,-0.630530,-0.660452,-0.754675,-0.640063,-0.654674,-0.762258
3658,-0.671409,-0.758133,-1.054720,-0.671037,-0.771827,-1.058862,-0.655172,-0.744954,-1.038246,-0.656714,...,-0.759677,-0.631244,-0.648311,-0.740164,-0.630530,-0.660452,-0.754675,-0.640063,-0.654674,-0.762258
3659,-0.671409,-0.205419,-0.149226,-0.671037,0.303475,0.278686,-0.655172,0.306712,-0.168906,-0.656714,...,-0.353555,-0.318821,-0.163995,-0.332818,-0.313751,-0.160126,-0.336957,-0.316229,-0.143280,-0.335113
3660,1.032235,0.900010,0.303521,1.312019,0.841125,0.724536,0.983205,0.832545,1.135103,1.262589,...,0.864812,1.243292,1.288951,0.889221,1.270144,1.340852,0.916195,1.302940,1.390903,0.946319


In [16]:
# x_s= x_scaled.to_pandas() ram using because of 2 data frames
# know converting from cudf to pandas and viceversa and manage ram effictivly by clearing the excess data in ram and gpu

In [17]:
del df
import gc
gc.collect()  # To e

202509

In [18]:
from cuml.decomposition import IncrementalPCA
import gc
incremental_pca = IncrementalPCA(0.95)

# Fit and transform the data in batches
batch_size = 10000  # Adjust based on your memory constraints
for i in range(0, x_scaled.shape[0], batch_size):
    end = min(i + batch_size, x_scaled.shape[0])
    incremental_pca.partial_fit(x_scaled[i:end])

df_pca_gpu = incremental_pca.transform(x_scaled)#cudf data frame

# Convert the result to a cuDF DataFrame for better readability
# df_pca_gpu = cudf.DataFrame(df_pca_gpu, columns=['PC1', 'PC2'])

# Display the first few rows of the PCA result
# print(df_pca_gpu.head())

# Clear unused variables to free up memory
# del x, x_scaled
# gc.collect()
# cp.get_default_memory_pool().free_all_blocks()

<ipython-input-18-fb56dcc972fd>:3: FutureWarning: Pass handle=0.95 as keyword args. From version 21.06, passing these as positional arguments will result in an error
  incremental_pca = IncrementalPCA(0.95)


In [19]:
print(type(df_pca_gpu))
# Output: <class 'cudf.core.dataframe.DataFrame'>

print(df_pca_gpu.shape)
# Output: (100000, <n_components>)

print(df_pca_gpu.dtypes)
# Output: float32 (for each of the n_components columns)

<class 'cudf.core.dataframe.DataFrame'>
(3662, 3662)
0       float64
1       float64
2       float64
3       float64
4       float64
         ...   
3657    float64
3658    float64
3659    float64
3660    float64
3661    float64
Length: 3662, dtype: object


In [20]:
df_pca_gpu

,0,1,2,3,4,5,6,7,8,9,...,3652,3653,3654,3655,3656,3657,3658,3659,3660,3661
0,-161.940819,120.476309,-38.199461,20.682979,11.226261,-46.194115,15.063510,-61.032234,-36.835332,20.361007,...,-1.791993e-15,-6.356207e-15,-6.744836e-15,7.731693e-15,-8.647901e-15,6.780544e-16,-6.200221e-15,-3.635237e-15,1.742761e-15,6.844663e-15
1,-102.299625,19.620283,-79.983283,-29.714447,-60.409552,5.458887,23.159045,16.107699,-58.805648,13.600965,...,-2.209665e-15,-2.013012e-15,-1.584375e-15,5.652277e-15,-3.495407e-15,4.875221e-15,-1.918968e-15,-6.237937e-16,2.944247e-15,-2.990890e-15
2,-126.321040,-97.736306,165.155347,58.378018,49.207038,20.094583,36.695850,10.232926,8.787072,14.727263,...,5.975197e-15,-5.270208e-15,1.241846e-14,-6.476704e-16,-4.266125e-15,-7.209125e-15,-1.096147e-15,-1.529852e-14,-7.271462e-15,-6.638481e-16
3,116.816613,-102.748307,7.470246,24.476956,-57.506910,-67.467815,-7.476084,-32.478109,-8.159653,-8.438463,...,8.928044e-17,2.432831e-15,-6.613189e-15,4.341394e-15,2.934712e-15,-1.465455e-15,2.812230e-16,9.995916e-15,-1.003906e-15,-5.314642e-15
4,-79.595642,329.494274,-11.559383,22.964598,-6.548903,-61.254008,62.221443,-122.801417,-28.495868,15.973453,...,-4.561026e-15,-1.069398e-15,-6.862116e-15,5.530595e-15,-8.666637e-16,1.096117e-15,-6.781697e-15,-5.393738e-15,1.560667e-15,-1.950176e-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3657,178.240284,-36.468862,10.815698,-13.219673,57.326222,-77.952501,-13.535276,2.197973,-8.781792,-10.454339,...,-4.056709e-15,6.619587e-15,5.633959e-15,-2.316474e-15,3.053540e-15,-3.967794e-15,4.088507e-15,1.850213e-14,4.899358e-15,5.079821e-15
3658,-171.004494,-77.010252,138.798167,49.857152,-42.314156,19.835959,27.958513,23.674863,45.173181,-21.992295,...,-2.559295e-16,-1.494303e-14,1.891327e-14,1.014925e-14,-1.478549e-14,-7.583825e-15,-1.207170e-15,-2.035004e-14,1.568689e-15,2.986010e-15
3659,91.508222,-81.830456,4.909027,15.550344,-51.931663,-64.292126,-23.290464,-20.037433,-5.339317,-16.984698,...,2.698879e-15,7.936085e-15,2.715756e-15,-7.542588e-15,5.239137e-16,1.085607e-14,8.085604e-15,-9.171026e-16,-5.464301e-15,-1.560974e-14
3660,-104.765324,166.178115,-32.220665,-4.404426,61.475480,-42.104218,12.780013,-37.686070,-8.975613,7.674883,...,2.440067e-15,1.451217e-14,2.304351e-15,2.430616e-14,1.259538e-15,-2.217463e-15,8.651679e-15,9.502744e-16,1.034818e-14,4.269047e-15


In [21]:
# Clear unused variables to free up memory
del x, x_scaled
gc.collect()
cp.get_default_memory_pool().free_all_blocks()

In [22]:
from cuml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_pca_gpu, y, test_size=0.2, random_state=42)

In [23]:
from cuml.svm import SVC
from cuml.metrics import accuracy_score


In [24]:
model= SVC(kernel='rbf')
model.fit(X_train,y_train)

SVC()

In [25]:
prediction=model.predict(X_test)

In [26]:
acc=accuracy_score(y_test,prediction)
acc #93% accuracy on binary classification

0.9330601096153259

In [27]:
print(type(y_test))

<class 'cudf.core.series.Series'>


In [28]:
print(type(prediction))

<class 'cudf.core.series.Series'>


In [29]:
Y_tes=y_test.to_pandas()
y_pred=prediction.to_pandas()

In [30]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(Y_tes,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       361
           1       0.93      0.94      0.93       371

    accuracy                           0.93       732
   macro avg       0.93      0.93      0.93       732
weighted avg       0.93      0.93      0.93       732



In [31]:
# try using and knowing aout roc and auroc to gain more insights

In [33]:
import joblib
joblib.dump(model,'model.pkl')

['model.pkl']

In [35]:
model_2=joblib.load('/content/model.pkl')

In [36]:
model_2.predict(X_test)

268     1
2054    1
921     1
762     0
599     1
       ..
3091    0
1599    0
2615    0
3102    1
2822    1
Length: 732, dtype: int64

In [37]:
model_2.score(X_test,y_test)

0.9330601096153259

In [ ]:
/content/drive/MyDrive/ ML data/02685f13cefd.png

In [46]:
img_exp=cv2.imread('/content/drive/MyDrive/ ML data/02685f13cefd.png')

In [58]:
def preprocess_image(image):
    from sklearn.preprocessing import StandardScaler
    # Resize the image to a fixed size (e.g., 150x150)
    resized_image = cv2.resize(image, (150, 150))
    # Flatten the image into a 1D array
    flattened_image = resized_image.flatten()
    df=pd.DataFrame(flattened_image)
    df=df.T
    scale=StandardScaler()
    df=scale.fit_transform(df)

    return df

In [59]:
df=preprocess_image(img_exp)

In [61]:
df.info()

AttributeError: 'numpy.ndarray' object has no attribute 'info'

In [60]:
df

array([[0., 0., 0., ..., 0., 0., 0.]])

In [62]:
from sklearn.decomposition import PCA

In [64]:
x_pca=PCA(0.95)
df=x_pca.fit_transform(df)

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_pca.py:543: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)


In [67]:
df.n_components

AttributeError: 'numpy.ndarray' object has no attribute 'n_components'